In [1]:
import numpy as np
import astropy.units as u
from astropy import time
from poliastro import iod
from poliastro.bodies import Sun
from poliastro.twobody import Orbit
from poliastro.util import time_range
import matplotlib.pyplot as plt

from astropy.coordinates import solar_system_ephemeris, get_body_barycentric_posvel
solar_system_ephemeris.set("jpl")

N = 50
date_launch = time.Time("2011-11-26 15:02", scale = 'utc')
date_arrival = time.Time("2012-08-06 05:17", scale = 'utc')
tof = date_arrival - date_launch
tof.to(u.h)
print(tof)

times_vector = time_range(date_launch, end=date_arrival, periods = N)
print(times_vector[:5])

rr_earth, vv_earth = get_body_barycentric_posvel("earth",times_vector)
print(rr_earth[:3])

rr_mars, vv_mars = get_body_barycentric_posvel("mars",times_vector)
print(rr_mars[:3])
print(vv_mars[:3])

# Compute the transfer orbit:
r0 = rr_earth[0].xyz
rf = rr_mars[-1].xyz
(va, vb), = iod.lambert(Sun.k, r0, rf,tof)

ss0_trans = Orbit.from_vectors(Sun, r0, va, date_launch)
ssf_trans = Orbit.from_vectors(Sun, rf, vb, date_arrival)

from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)

from poliastro.plotting import OrbitPlotter3D
from poliastro.bodies import Earth, Mars

# I like color
color_earth0 = '#3d4cd5'
color_earthf = '#525fd5'
color_mars0 = '#ec3941'
color_marsf = '#ec1f28'
color_sun = '#ffcc00'
color_orbit = '#888888'
color_trans = '#444444'

frame = OrbitPlotter3D()

frame.set_attractor(Sun)

frame.plot_trajectory(rr_earth, label=Earth, color=color_earth0)
frame.plot_trajectory(rr_mars, label=Mars, color=color_marsf)

frame.plot_trajectory(ss0_trans.sample(times_vector), label="MSL trajectory", color=color_trans)

frame.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)
frame.show(title="MSL Mission: from Earth to Mars")

253.59376157407408
['2011-11-26 15:02:00.000' '2011-12-01 19:14:33.082'
 '2011-12-06 23:27:06.163' '2011-12-12 03:39:39.245'
 '2011-12-17 07:52:12.327']
[( 64600643.37167563,   1.21424866e+08,  52640047.33041222),
 ( 52175250.21264037,   1.26254284e+08,  54733247.42732787),
 ( 39319701.40598051,   1.30036609e+08,  56373071.6065251 )] km
[( -1.23149631e+08,   1.90752511e+08,  90809654.2669948 ),
 ( -1.31992428e+08,   1.86383187e+08,  89044491.25204735),
 ( -1.40598005e+08,   1.81677346e+08,  87118570.32883616)] km
[(-1730626.66251077, -811069.96095538, -325255.37513281),
 (-1686163.26853493, -877100.53950512, -356742.77622963),
 (-1638971.32577256, -941103.98130274, -387374.07466148)] km / d
